In [1]:
import pandas as pd
import numpy as np
brdiff = pd.read_table('Historical marker crude prices_csv.csv',delimiter =';')
brdiff = brdiff.stack().str.replace(',','.').unstack()
brdiff=brdiff.fillna(0)
print(brdiff)

brdiff.dtypes
brdiff =brdiff.iloc[:,1:11].astype(float)

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

brdiff["Dated Brent scaled"] = scaler.fit_transform(brdiff["Dated Brent"].to_frame())

import matplotlib.pyplot as plt  

from sklearn import linear_model
import statsmodels.api as sm
#X = df[['Interest_Rate','Unemployment_Rate']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
#Y = df['Stock_Index_Price']
 
# with sklearn
#regr = linear_model.LinearRegression()
#regr.fit(X, Y)

AbsB=brdiff["Dated Brent"].max()
brdiff["Dated Brent norm"] = brdiff["Dated Brent"].divide(AbsB)
print(brdiff["Dated Brent norm"])

X = brdiff[['Dated Brent','Maya USGC']]
Y = brdiff['Bonny light']
regr = linear_model.LinearRegression()
regr.fit(X, Y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

New_DB = 64.887
New_Maya = -5.675
print ('Predicted BL: \n', regr.predict([[New_DB, New_Maya]]))

           Date Dated Brent Bonny light Mars    WTI Maya Europe Maya USGC  \
0    01.01.2000        25.5           0    0      0           0         0   
1    01.02.2000        27.9           0    0      0           0         0   
2    01.03.2000        27.3           0    0      0           0         0   
3    01.04.2000        22.6           0    0      0           0         0   
4    01.05.2000        27.6           0    0      0           0         0   
..          ...         ...         ...  ...    ...         ...       ...   
231  01.04.2019        71.3         1.4  5.1   -7.4        -9.8      -6.5   
232  01.05.2019        71.1         1.9  5.9  -10.3       -10.1      -8.4   
233  01.06.2019        64.1         1.7  4.8   -9.4        -8.1      -5.2   
234  01.07.2019        64.0         1.4  4.5   -6.5        -6.9      -4.1   
235  01.08.2019        59.0         1.3  2.3   -4.2       -12.6      -8.0   

     ESPO Urals NWE Urals MED Dubai  
0       0      -0.5      -0.5  -2.2  

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.dpi'] = 85

def f(db, m):
#def f(db):  
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    x = brdiff["Dated Brent"]
    y=regr.predict([[db, m]])
    #y=regr.predict([[db]])
    
    ax1.scatter(brdiff["Dated Brent"],brdiff["Bonny light"])
    ax1.scatter(db,y, s=200)
    k1= brdiff["Dated Brent"]
    k2= brdiff["Maya USGC"]
    ax2.scatter(brdiff["Bonny light"], regr.intercept_+ regr.coef_[0]*k1 + regr.coef_[1]*k2 )
    #ax2.scatter(brdiff["Bonny light"], regr.intercept_+ regr.coef_[0]*k1 )
    ax2.set_xlim([0, 4])
    ax2.set_ylim([0, 4])
    ax1.set_xlabel("Brent price")
    ax1.set_ylabel("Bonny Light diff")
    ax2.set_xlabel("Historic Bonny Light diff")
    ax2.set_ylabel("Predicted Bonny Light diff")
    plt.subplots_adjust(wspace = 0.4)
    plt.show()
    
    
    
    print("predicted Bonny Light is", y )

#interactive_plot = interactive(f, db=(10,150,1))
interactive_plot = interactive(f, db=(10,150,1), m=(-6, 6, 0.1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


interactive(children=(IntSlider(value=80, description='db', max=150, min=10), FloatSlider(value=0.0, descripti…

In [8]:

X = brdiff[['Dated Brent','WTI','Urals NWE','Urals MED']]
Y = brdiff['Mars']
regr = linear_model.LinearRegression()
regr.fit(X, Y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

New_DB = 64.887
New_WTI = -7.81
New_UralsNWE = -0.7
New_UralsMED = -0.1
print ('Predicted Mars: \n', regr.predict([[New_DB, New_WTI,New_UralsNWE,New_UralsMED ]]))

Intercept: 
 -0.5162759538376632
Coefficients: 
 [-0.01393699 -0.94378365  2.48133925 -1.51308904]
                            OLS Regression Results                            
Dep. Variable:                   Mars   R-squared:                       0.909
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                     576.4
Date:                Thu, 05 Sep 2019   Prob (F-statistic):          6.76e-119
Time:                        14:42:54   Log-Likelihood:                -477.06
No. Observations:                 236   AIC:                             964.1
Df Residuals:                     231   BIC:                             981.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [5]:
X = brdiff[['Dated Brent']]
Y = brdiff['Mars']
regr = linear_model.LinearRegression()
regr.fit(X, Y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

New_DB = 64.887
New_WTI = -7.81
New_UralsNWE = -0.7
New_UralsMED = -0.1
print ('Predicted Mars: \n', regr.predict([[New_DB ]]))

Intercept: 
 -6.826578526862411
Coefficients: 
 [0.09621173]
                            OLS Regression Results                            
Dep. Variable:                   Mars   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     69.86
Date:                Thu, 05 Sep 2019   Prob (F-statistic):           5.69e-15
Time:                        14:42:16   Log-Likelihood:                -728.98
No. Observations:                 236   AIC:                             1462.
Df Residuals:                     234   BIC:                             1469.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [9]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.dpi'] = 100

def f(db, m1,m2,m3):
 
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    x = brdiff["Dated Brent"]
    y=regr.predict([[db, m1,m2,m3]])
    
    
    ax1.scatter(brdiff["Dated Brent"],brdiff["Mars"])
    ax1.scatter(db,y, s=200)
    k1= brdiff["Dated Brent"]
    k2= brdiff["WTI"]
    k3= brdiff["Urals NWE"]
    k4= brdiff["Urals MED"]
    
    ax2.scatter(brdiff["Mars"], regr.intercept_+ regr.coef_[0]*k1 + regr.coef_[1]*k2 + regr.coef_[2]*k3 + regr.coef_[3]*k4)
    ax2.set_xlim([-10, 10])
    ax2.set_ylim([-10, 10])
    ax1.set_xlabel("Brent price")
    ax1.set_ylabel("Mars diff")
    ax2.set_xlabel("Historic Mars diff")
    ax2.set_ylabel("Predicted Mars diff")
    plt.subplots_adjust(wspace = 0.4)
    plt.show()
    
    
    
    print("predicted Mars is", y )


interactive_plot = interactive(f, db=(10,150,1), m1=(-10, 10, 0.1), m2=(-6, 6, 0.1), m3=(-6, 6, 0.1))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

interactive(children=(IntSlider(value=80, description='db', max=150, min=10), FloatSlider(value=0.0, descripti…

In [7]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.dpi'] = 100

def f(db):
 
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    x = brdiff["Dated Brent"]
    y=regr.predict([[db]])
    
    
    ax1.scatter(brdiff["Dated Brent"],brdiff["Mars"])
    ax1.scatter(db,y, s=200)
    k1= brdiff["Dated Brent"]
    k2= brdiff["WTI"]
    k3= brdiff["Urals NWE"]
    k4= brdiff["Urals MED"]
    
    ax2.scatter(brdiff["Mars"], regr.intercept_+ regr.coef_[0]*k1 )
    ax2.set_xlim([-10, 10])
    ax2.set_ylim([-10, 10])
    ax1.set_xlabel("Brent price")
    ax1.set_ylabel("Mars diff")
    ax2.set_xlabel("Historic Mars diff")
    ax2.set_ylabel("Predicted Mars diff")
    plt.subplots_adjust(wspace = 0.4)
    plt.show()
    
    
    
    print("predicted Mars is", y )


interactive_plot = interactive(f, db=(10,150,1))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

interactive(children=(IntSlider(value=80, description='db', max=150, min=10), Output(layout=Layout(height='450…

In [2]:
X = brdiff[['Dated Brent','Urals MED']].iloc[-150:]
Y = brdiff['Maya Europe'].iloc[-150:]
regr = linear_model.LinearRegression()
regr.fit(X, Y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)



Intercept: 
 -7.126254813795303
Coefficients: 
 [-0.03086658  0.45521668]
                            OLS Regression Results                            
Dep. Variable:            Maya Europe   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     5.500
Date:                Fri, 06 Sep 2019   Prob (F-statistic):            0.00497
Time:                        08:34:49   Log-Likelihood:                -418.03
No. Observations:                 150   AIC:                             842.1
Df Residuals:                     147   BIC:                             851.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [10]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.dpi'] = 150

def f(db,ural):
 
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    x = brdiff["Dated Brent"]
    y=regr.predict([[db,ural]])
    
    
    ax1.scatter(brdiff["Dated Brent"],brdiff["Maya Europe"])
    ax1.scatter(db,y, s=200)
    k1= brdiff["Dated Brent"]
    k4= brdiff["Urals MED"]
    
    ax2.scatter(brdiff["Maya Europe"], regr.intercept_+ regr.coef_[0]*k1 )
    ax2.set_xlim([-20, 1])
    ax2.set_ylim([-20, 1])
    ax1.set_xlabel("Brent price")
    ax1.set_ylabel("Maya Europe diff")
    ax2.set_xlabel("Historic Maya Europe  diff")
    ax2.set_ylabel("Predicted Maya Europe  diff")
    plt.subplots_adjust(wspace = 0.4)
    plt.show()
    
    
    
    print("predicted Maya Europe  is", y )


interactive_plot = interactive(f, db=(10,150,1),ural =(-5,5,0.1))
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

interactive(children=(IntSlider(value=80, description='db', max=150, min=10), FloatSlider(value=0.0, descripti…